# Thermodynamic databases

## Loading the database

Reaktoro requires *thermodynamic databases* to calculate the *thermodynamic properties of chemical species and reactions*. These properties are necessary when performing chemical equilibrium and kinetics calculations.

Examples of such **thermodynamic properties for a species $i$** include:

* standard molar Gibbs energy, $G_i^\circ$
* standard molar Helmholtz energy, $A_i^\circ$
* standard molar enthalpy, $H_i^\circ$
* standard molar internal energy, $U_i^\circ$
* standard molar entropy, $S_i^\circ$
* standard molar volume, $V_i^\circ$
* standard molar heat capacity (constant pressure), $C_{P,i}^\circ$
* standard molar heat capacity (constant volume), $C_{V,i}^\circ$

The above standard thermodynamic properties of the species can be used to calculate **properties for a reaction $m$**:

* equilibrium constant of reaction, $K_m$
* standard Gibbs energy of reaction, $\Delta G_m^\circ$
* standard enthalpy of reaction, $\Delta H_m^\circ$

Reaktoro currently supports thermodynamic databases from the following providers:

* [PHREEQC](https://www.usgs.gov/software/phreeqc-version-3)
* [ThermoFun](https://thermohub.org/thermofun/thermofun/)
* [SUPCRT](https://www.sciencedirect.com/science/article/pii/009830049290029Q)
* [SUPCRTBL](https://models.earth.indiana.edu/supcrtbl.php)
* [NASA](https://www1.grc.nasa.gov/research-and-engineering/ceaweb/)

They are loaded via the classes

* [PhreeqcDatabase](https://reaktoro.org/api/classReaktoro_1_1PhreeqcDatabase.html)
* [ThermoFunDatabase](https://reaktoro.org/api/classReaktoro_1_1ThermoFunDatabase.html)
* [SupcrtDatabase](https://reaktoro.org/api/classReaktoro_1_1SupcrtDatabase.html)
* [NasaDatabase](https://reaktoro.org/api/classReaktoro_1_1NasaDatabase.html)

derived from the [Database](https://reaktoro.org/api/classReaktoro_1_1Database.html) class.

There are two ways how different thermodynamic databases from distinct providers can be loaded in Reaktoro. 

* Many *standard databases* are embedded into Reaktoro and can be loaded (e.g., for PHREEQC databases) as

~~~python 
db = PhreeqcDatabase("phreeqc.dat")
~~~

* A *local user database* file (written in a correct format) can be specified manually:

~~~python 
db = PhreeqcDatabase.fromFile("/path-to-file/local-phreeqc.dat")
~~~


## Inspecting the database

To inspect thermodynamic databases, we will consider the [cemdata18](https://www.empa.ch/web/s308/cemdata) database supported in [ThermoFun](https://thermohub.org/thermofun/thermofun/).

In [1]:
from reaktoro import *

db = ThermoFunDatabase("cemdata18")

### Inspecting the species in the database

Below us inspect all species in this database object and print its name, chemical formula, and molar mass:

In [2]:
print(f"{'Species':<20}{'Formula':<60}{'Molar Mass (kg/mol)':<20}")
for species in db.species():
    print(f"{species.name():<20}{species.formula().str():<60}{species.molarMass():<20.6f}")

Species             Formula                                                     Molar Mass (kg/mol) 
5CA                 (CaO)1.25(SiO2)1(Al2O3)0.125(H2O)1.625                      0.172201            
5CNA                (CaO)1.25(SiO2)1(Al2O3)0.125(Na2O)0.25(H2O)1.375            0.183192            
Al(SO4)+            Al(SO4)+                                                    0.123046            
Al(SO4)2-           Al(SO4)2-                                                   0.219111            
Al+3                Al+3                                                        0.026980            
AlO+                AlO+                                                        0.042980            
AlO2-               AlO2-                                                       0.058981            
AlO2H@              AlO2H@                                                      0.059988            
AlOH+2              Al(OH)+2                                                    0.043988   

To print those **species that contain certain chemical list of elements**:

In [3]:
for species in db.species().withElements("H O C Ca"):
    print(species.name())

Arg
CH4
CH4@
CO2
CO2@
CO3-2
Ca(CO3)@
Ca(HCO3)+
Ca+2
CaOH+
Cal
Gr
H+
H2
H2@
H2O
H2O@
HCO3-
Lim
O2
O2@
OH-
Portlandite


To collect **only gaseous species** from a database, the function below can be used:

In [4]:
for species in db.speciesWithAggregateState(AggregateState.Gas):
    print(species.name())

CH4
CO2
H2
H2O
H2S
N2
O2


**TASK 1**: output only aqueous species with C, H, O elements

In [5]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #
for species in db.speciesWithAggregateState(AggregateState.Gas).withElements("C H O"):
    print(species.name())

CH4
CO2
H2
H2O
O2


## Inspecting the elements in the database

To check all the **chemical elements** available in the database:

In [6]:
print(f"{'Symbol':<10}{'Molar Mass (kg/mol)':<10}")
for element in db.elements():
    print(f"{element.symbol():<10}{element.molarMass():<10.6f}")

Symbol    Molar Mass (kg/mol)
Al        0.026982  
Ca        0.040078  
H         0.001008  
O         0.015999  
Si        0.028086  
Na        0.022990  
S         0.032067  
C         0.012011  
Mg        0.024305  
Cl        0.035453  
Fe        0.055845  
Sr        0.087620  
K         0.039098  
N         0.014007  


## Inspecting species' and elements' parameters in the database

A thermodynamic database contains* model parameters* for the evaluation of standard thermodynamic properties of species and/or reactions (e.g., standard Gibbs energies, equilibrium constants). These properties can be accessed by fetching a particular species from the database and calling a corresponding functions to output, e.g., charge and dissociation of CaCl<sub>2</sub>:

In [7]:
species = db.species().getWithName("Ca(SO4)@")
print("Name of Ca(SO4)@           : ", species.name())
print("Formula of Ca(SO4)@        : ", species.formula())
print("Charge of Ca(SO4)@         : ", species.charge())
print("Aggregate state of Ca(SO4)@: ", species.aggregateState())

Name of Ca(SO4)@           :  Ca(SO4)@
Formula of Ca(SO4)@        :  CaSO4@
Charge of Ca(SO4)@         :  0.0
Aggregate state of Ca(SO4)@:  AggregateState.Aqueous


**TASK 2**: use SUPCRT database to 
* fetch species having solid aggregate state and coposed of Ca S C H O elements and 
* inspect the properties of Pyrite, i.e., formula, molar mass, charge, as well as elements, which Pyrrhotite is constructed from.

In [8]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #
db = SupcrtDatabase("supcrtbl")
print("List of all solids in database supcrtbl composed of Ca S C H O elements:")
for minerals in db.speciesWithAggregateState(AggregateState.Solid).withElements("Ca S C H O"):
    print(minerals.name())

species = db.species().getWithName("Pyrite")

print(f"Formula of Pyrrhotite   : {species.formula()}")
print(f"Molar mass of Pyrrhotite: {species.molarMass():4.4e} kg/mol")
print(f"Charge of Pyrrhotite    : {species.charge()}")
print(f"Elemental composition   : {species.elements().symbols()}")

List of all solids in database supcrtbl composed of Ca S C H O elements:
Lime
Anhydrite
Diamond
Graphite
Sulphur
Calcite
Aragonite
Formula of Pyrrhotite   : FeS2
Molar mass of Pyrrhotite: 1.1998e-01 kg/mol
Charge of Pyrrhotite    : 0.0
Elemental composition   : ['Fe', 'S']
